---
# <div align="center"><font color='green'>  </font></div>
# <div align="center"><font color='green'> COSC 2673/2793 | Machine Learning  </font></div>
## <div align="center"> <font color='green'> **Example: Week10 Lecture QandA**</font></div>
---

# Traditional Computer Vision

First lets have a look at some of the common operations used in traditional computer vision. Namely, Convolutions and feture extractors.

Lets first read an image of my Dog: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from skimage.feature import hog
from skimage import data, exposure
from skimage.io import imread
from skimage.color import rgba2rgb, rgb2gray

image = rgba2rgb(imread('shady.png', as_gray=False, plugin=None))

# Plot image
plt.imshow(image, cmap=plt.cm.gray)
plt.axis('off')
plt.show()

Lets say we want to make the above image smooth (blur). In computer vision we do this through using the convolution operation. To smooth we need to use a gaussian kernal. We need to set the Standard deviation (sigma) for Gaussian kernel which will controll the smoothing. Large sigma will blur the image more. 

In [ ]:
from skimage.filters import gaussian

smooth_image_1 = gaussian(image, sigma=1, multichannel=True)
smooth_image_10 = gaussian(image, sigma=10, multichannel=True)

# Plot image
plt.figure(figsize=(18,6))
plt.subplot(1,3,1)
plt.axis('off')
plt.imshow(image)
plt.title("Original Image")

plt.subplot(1,3,2)
plt.imshow(smooth_image_1)
plt.title("Gaussian sigma = 1")
plt.axis('off')
plt.subplot(1,3,3)
plt.imshow(smooth_image_10)
plt.title("Gaussian sigma = 10")
plt.axis('off')
plt.show()

In computer vision , edge detection is often used to reduce the amount of information in an image. Lets try this.

In [ ]:
from skimage import filters

grayscale = rgb2gray(image)
smooth_image_1 = gaussian(grayscale, sigma=2)
edge_sobel_h = filters.sobel_h(smooth_image_1)
edge_sobel_v = filters.sobel_v(smooth_image_1)

# Plot image
plt.figure(figsize=(18,6))
plt.subplot(1,3,1)
plt.axis('off')
plt.imshow(image)
plt.title("Original Image")

plt.subplot(1,3,2)
plt.imshow(edge_sobel_h, cmap=plt.cm.coolwarm)
plt.axis('off')
plt.title("Horizontal edges")
plt.subplot(1,3,3)
plt.imshow(edge_sobel_v, cmap=plt.cm.coolwarm)
plt.axis('off')
plt.title("Vertical edges")
plt.show()

Computer vision researchers have developed useful feature representations that are useful for many applications. One of them is the HOG features.
We will now extract HOG features (Histogram of Gradients). More information on HoG features can be found at [website](https://www.learnopencv.com/histogram-of-oriented-gradients/). There are several other feature types availble with sk-image see [page](https://scikit-image.org/docs/dev/api/skimage.feature.html)

In [ ]:
hog_features, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, .1))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()

In [ ]:
print("Image Shape: ", image.shape, "Flattened shape : ", np.prod(image.shape))
print("HoG feature Shape: ", hog_features.shape, "Flattened shape : ", np.prod(hog_features.shape))

Now the `hog_features` can be used as the input to ML algorithm.

# Deep Learning (Convolutional Neural Networks)

In deep learning we skip this feature construction stage by using a deep network with many layers. The process to develop a deep network will be very similer to how we developed the shallow network. The main difference would be in the architecture selection. Usually instead of selecting a totally new architechture, one would investigate the existing architectures in literature and use it as the base model. Then we can modify it based on the observations we make (e.g. overfitting and underfitting). 


Lets try to solve the problem from last weeks lecture QandA Example. The task was to predict the face orientation given an image of a human face. 

Lets start by setting up the data. This is exactly similer to what we did last week. 

In [ ]:
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

import pandas as pd

import zipfile
with zipfile.ZipFile('./faces_TM.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

val_persons = ['sz24', 'megak', 'night', 'choon', 'kawamura']

from PIL import Image
import glob
image_list = []
for filepath in glob.glob('./faces_TM/*/*.jpg', recursive=True): #assuming gif
    filename = filepath.split("/")[-1]
    person = filepath.split("/")[-2]
    label = filename.split("_")[1]
    val_train = person in val_persons
    image_list.append((filepath, label, int(val_train)))
    
# Create a data frame
data = pd.DataFrame(data=image_list, columns=['image_path', 'label', 'isVal'])

d = {'left':0, 'right':1, 'straight':2, 'up':3}
data['labels_num'] = data['label'].map(d, na_action='ignore')

train_df = data[data['isVal']==0].reset_index()
validation_df = data[data['isVal']==1].reset_index()
print('Train size: {}, Val size: {}'.format(train_df.shape[0], validation_df.shape[0] ) )
N_train_images = train_df.shape[0]
N_val_images = validation_df.shape[0]

train_df.to_csv('TrainData.csv')
validation_df.to_csv('ValData.csv')

Setup data loaders as we did last week

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')
val_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')

batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='./',
        x_col="image_path",
        y_col="label",
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_dataframe(
        dataframe=validation_df,
        directory='./',
        x_col="image_path",
        y_col="label",
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')

## Setting up base model

We need to select a base model for this task. What would be a good model. According to the lecture we can select the laNet model. This is a simple model that can operate on images that are small. Lets implement it. We have made few modifications here to the original lanet architechture. 

In [ ]:
reg_lambda = 0.001
OUTPUT_CLASSES = 4

model_leNet = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 3)),
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x[:,:,:,0], -1, name=None)),
    
    
    tf.keras.layers.Conv2D(32, (3, 3), kernel_regularizer=tf.keras.regularizers.l2(reg_lambda)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    
    tf.keras.layers.Conv2D(32, (3, 3), kernel_regularizer=tf.keras.regularizers.l2(reg_lambda)),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(64),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(OUTPUT_CLASSES, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(reg_lambda))
])

Now lets train this model (same as last week)

In [ ]:
sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_leNet.compile(optimizer=sgd,
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['categorical_accuracy'])

history = model_leNet.fit_generator(train_generator, 
                                    validation_data = validation_generator, 
                                    epochs=100, verbose=0)

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'r--')
plt.plot(history.history['val_loss'], 'b--')
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.legend(['train', 'val'], loc='upper left')

plt.subplot(1,2,2)
plt.plot(history.history['categorical_accuracy'], 'r--')
plt.plot(history.history['val_categorical_accuracy'], 'b--')
plt.xlabel("epochs")
plt.ylabel("Accuracy")
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.show()

## Data Augmentation

If the model is overfirring we can use all the techniques we discussed last week to companseate for that. In addition a common technique used in deep CNN for regularization is data augmentation. 

In data augmentation, we randomly perterbate the original dataset to form a larger dataset and use that for training. 

The keras data grnrrator can do this for us. However we need to pick the data augmentation techniques that are appropriate for the task. Lets see how this is done. We apply data augmentation only for trainig data (not for validation or test)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last',
                                  rotation_range=15, width_shift_range=0.2,
                                  height_shift_range=0.2)
val_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')

batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='./',
        x_col="image_path",
        y_col="label",
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_dataframe(
        dataframe=validation_df,
        directory='./',
        x_col="image_path",
        y_col="label",
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
model_leNet_aug = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 3)),
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x[:,:,:,0], -1, name=None)),
    
    
    tf.keras.layers.Conv2D(32, (3, 3), kernel_regularizer=tf.keras.regularizers.l2(reg_lambda)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    
    tf.keras.layers.Conv2D(32, (3, 3), kernel_regularizer=tf.keras.regularizers.l2(reg_lambda)),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(OUTPUT_CLASSES, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(reg_lambda))
])

model_leNet_aug .compile(optimizer=sgd,
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['categorical_accuracy'])

history = model_leNet_aug .fit_generator(train_generator, 
                                    validation_data = validation_generator, 
                                    epochs=150, verbose=0)

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'r--')
plt.plot(history.history['val_loss'], 'b--')
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.legend(['train', 'val'], loc='upper left')

plt.subplot(1,2,2)
plt.plot(history.history['categorical_accuracy'], 'r--')
plt.plot(history.history['val_categorical_accuracy'], 'b--')
plt.xlabel("epochs")
plt.ylabel("Accuracy")
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.show()